Install the cdsapi package to download the data. We will need xarray to concatenate the data. netCDF4, eccodes, cfgrib, ecmwflibs, and netcdf4 are needed to open the GRIB file and also convert to NetCDF.

When you wish to execute shell commands in a jupyter notebook, start the command with "!". I am using a virtual environment to run this notebook, so it won't install the packages globally.

In [1]:
! python -m pip install cdsapi xarray pandas eccodes cfgrib ecmwflibs netcdf4 ecmfw

ERROR: Could not find a version that satisfies the requirement ecmfw (from versions: none)
ERROR: No matching distribution found for ecmfw

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
! pip install "cdsapi>=0.7.2"


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import cdsapi
import xarray as xr
from timeit import default_timer as timer

In [4]:
client = cdsapi.Client()

2025-04-16 15:04:45,295 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-04-16 15:04:45,298 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.


### Download a single field with wget.

Possible using the HTTP Byte-Range request feature.

Example: Download temperature at 2m at step=24h from the 00 UTC HRES forecast on 25 January 2022.

First download the associated index file by substituting the '.grib2' extension with '.index' in the URL.

```
wget [ROOT]/20240301/00z/ifs/0p25/oper/20240301000000-24h-oper-fc.index
```

Inspect the index file and look for the entry for 2m temperature ('param': '2t')

```
...
{"domain": "g", "date": "20240301", "time": "0000", "expver": "0001", "class": "od", "type": "fc", "stream": "oper", "step": "24", "levtype": "sfc", "param": "2t", "_offset": 17459800, "_length": 609046}
...
```

Use the values of "_offset" and "_length" keys for this record to construct the start bytes and end bytes:

```
start_bytes = __offset
end_bytes == __offset + __length - 1
```

The "_offset" and "_length" values of a specific field will change from forecast run to forecast run.  It is necessary to redo this computation for each download.

Use the start_bytes and end_bytes values calculated to pass the range of bytes to be downloaded to wget, this time for the ".grib2" file:

```
wget [ROOT]/20240301/00z/ifs/0p25/oper/20240301000000-24h-oper-fc.grib2 --header="Range: bytes=17459800-18068845"
```

Alternatively, curl can be used:

```
curl --range  17459800-18068845 [ROOT]/20240301/00z/ifs/0p25/oper/20240301000000-24h-oper-fc.grib2 --output 2t.grib2
```

In [14]:
! curl -o ./curl.grib2 "https://data.ecmwf.int/forecasts/20250418/00z/ifs/0p25/oper/20250418000000-0h-oper-fc.grib2"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0  113M    0  254k    0     0   148k      0  0:12:59  0:00:01  0:12:58  149k
  2  113M    2 2526k    0     0   965k      0  0:02:00  0:00:02  0:01:58  967k
  4  113M    4 5278k    0     0  1454k      0  0:01:19  0:00:03  0:01:16 1456k
  7  113M    7 8382k    0     0  1815k      0  0:01:03  0:00:04  0:00:59 1817k
  9  113M    9 10.9M    0     0  1986k      0  0:00:58  0:00:05  0:00:53 2239k
 11  113M   11 13.4M    0     0  2075k      0  0:00:55  0:00:06  0:00:49 2744k
 14  113M   14 16.1M    0     0  2165k      0  0:00:53  0:00:07  0:00:46 2791k
 16  113M   16 18.8M    0     0  2242k      0  0:00:51  0:00:08  0:00:43 2816k
 19  113M   19 21.8M    0     0  2322k      0  0:00

In [15]:
! curl -o ./index.grib2 "https://data.ecmwf.int/forecasts/20250418/00z/ifs/0p25/oper/20250418000000-0h-oper-fc.index"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 34686  100 34686    0     0  20002      0  0:00:01  0:00:01 --:--:-- 20049


In [8]:
root = 'https://data.ecmwf.int/forecasts/'
yyyymmdd = '20250418'
hh = '00' # values are 00, 06, 12, 18
model = 'ifs'
resol = '0p25' # 0.25 degree
stream = 'oper' # high-resolution forecast
step = '0' # 0h to 144h by 3h and 150h to 240h by 6h
unit = 'h'
type = 'fc' # forecast
format = 'grib2' # or index

path = root + yyyymmdd + "/" + hh + "z/" + model + "/" + resol + "/" + stream + "/" + yyyymmdd + hh + "0000-" + step + unit + "-" + stream + "-" + type + "." + format
print(path)

https://data.ecmwf.int/forecasts/20250418/00z/ifs/0p25/oper/20250418000000-0h-oper-fc.grib2


```
[ROOT]/[yyyymmdd]/[HH]z/[model]/[resol]/[stream]/[yyyymmdd][HH]0000-[step][U]-[stream]-[type].[format]
```

In [24]:
# Open the GRIB file. Using 'with' will make sure to close the file before execution leaves the 'with' block.
grib_filename = '20250412000000-0h-oper-fc.grib2'
open = xr.open_dataset(grib_filename, engine='cfgrib', backend_kwargs={'filter_by_keys': {'shortName': '2t'}})

In [25]:
lat = 47.9 + 0.1
long = 106.9 + 0.1
kelvin = open['t2m'].sel(latitude=lat, longitude=long).data 
fahrenheit = (kelvin - 273.15) * 9/5 + 32
print(fahrenheit)

16.86593750000004


As of writing this, due to the recent CDS server migrations and slow queues for downloading the data, we can only download a dataset one month at a time. Thus, we will have to loop through every month and year we wish to download. **Important**: Make sure to change the range in 'years_list' to the decades you wish to download. For example, if you're downloading 1951 - 1960, then the range would be range(1951, 1961). In this notebook, I decided to download city data from Ulaanbaatar, Mongolia. The coordinates are 30.2672° N, 97.7431° W, respectively, or 30.3 and -97.7, respectively. **Important**: Since our dataset is specific to land data, it's likely that if the region includes any oceans or large bodies of water, the temperature recorded at those pixels will be inaccurate or missing.

The link to the dataset: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land?tab=overview 

Dataset name: ERA5-Land hourly data from 1950 to present

In [26]:
years_list = [str(year) for year in range(2024, 2025)]
leap_years = [str(year) for year in range(2024, 2025, 4)]

make_two_digits = ['01', '02', '03', '04', '05', '06', '07', '08', '09'] # API accepts single digit days in this format

thirty_one = make_two_digits + [str(year) for year in range(10, 32)]
thirty = make_two_digits + [str(year) for year in range(10, 31)]
feb_days_leap = make_two_digits + [str(year) for year in range(10, 30)]
feb_days_non_leap = make_two_digits + [str(year) for year in range(10, 29)]

months_list = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11'] # API aceepts months in this format
month_days = {
	'01':thirty_one,
	'02':feb_days_non_leap,
	'03':thirty_one,
	'04':thirty,
	'05':thirty_one,
	'06':thirty,
	'07':thirty_one,
	'08':thirty_one,
	'09':thirty,
	'10':thirty_one,
	'11':thirty,
	'12':thirty_one,
}

# Currently downloading data for a model for Mongolia. These are the coords for its capital Ulaanbaatar.
city_long = 106.9
city_lat = 47.9

# Region boundaries for the capital. Will produce a 5x5 grid of values each time step. The spatial resolution of the dataset is 0.1° x 0.1°.
bottom_lat = str(round(city_lat - 0.2, 1))
top_lat = str(round(city_lat + 0.2, 1))
left_long = str(round(city_long - 0.2, 1))
right_long = str(round(city_long + 0.2, 1))

print(bottom_lat, top_lat, left_long, right_long)
print(leap_years)
print(thirty_one)
print(thirty)
print(feb_days_leap)
print(feb_days_non_leap)

47.7 48.1 106.7 107.1
['2024']
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30']
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29']
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28']


In [27]:
yearly_dataset = []

# IMPORTANT: Change 'ds_path' to wherever you originally store the data
ds_path = 'C:/Users/sguti/preprocessing/Scorched Earth Data/'

for year in years_list:
    # Name of the NetCDF file to convert to after concatenating all monthly datasets
    nc_filename = ds_path + 'Mongolia_2D_ERA5_' + year + '.nc'

    start = timer()

    for month in months_list:
        # Take care of leap years
        if month == '02' and year in leap_years:
            month_days['02'] = feb_days_leap
        else:
            month_days['02'] = feb_days_non_leap
        # Name of the raw monthly GRIB file
        grib_filename = ds_path + 'Mongolia_2D_ERA5_' + year + '_' + month + '.grib'
        dataset = 'reanalysis-era5-land'
        request = {
            'variable': [
                '2m_temperature', 
                '2m_dewpoint_temperature', 
                'surface_pressure', 
                '10m_u_component_of_wind', 
                '10m_v_component_of_wind'
            ],
            'year': year,
            'month': month,
            'day': month_days[month],
            'time': [
                '00:00', '01:00', '02:00',
                '03:00', '04:00', '05:00',
                '06:00', '07:00', '08:00',
                '09:00', '10:00', '11:00',
                '12:00', '13:00', '14:00',
                '15:00', '16:00', '17:00',
                '18:00', '19:00', '20:00',
                '21:00', '22:00', '23:00',
            ],
            'data_format': 'grib',
            'download_format': 'unarchived',
            'area': [
                top_lat, left_long, bottom_lat, right_long,
            ],
        }

        client.retrieve(dataset, request, grib_filename)

        # Open the GRIB file. Using 'with' will make sure to close the file before execution leaves the 'with' block.
        with xr.open_dataset(grib_filename) as ds:
            # The GRIB file data contains mostly empty labels [number, surface, valid_time]. We'll drop them before we concatenate with the yearly dataset to save space.
            monthly_dataset = ds.drop(['number', 'surface', 'valid_time'], dim=None)
            if len(yearly_dataset) == 0:
                # If yearly_dataset is empty, there's nothing to concatenate with
                yearly_dataset = monthly_dataset
            else: 
                # We will concatenate the two datasets along the time dimension.
                yearly_dataset = xr.concat([yearly_dataset, monthly_dataset], dim="time")

    end = timer()

    print("Elapsed time in minutes for a year to download and process:", ((end - start) / 60))

    # Finally, save the yearly_dataset to a NetCDF file. While GRIB is the native format, GRIB data is generally “messier” than data in a self-describing format, such as NetCDF. 
    print("Storing to... ", nc_filename)
    yearly_dataset.to_netcdf(nc_filename)
    yearly_dataset = []

2025-04-16 16:12:04,636 INFO Request ID is e8d2a780-1d67-4f0e-90b2-fad1ec5459de
2025-04-16 16:12:04,828 INFO status has been updated to accepted
2025-04-16 16:12:13,799 INFO status has been updated to running


KeyboardInterrupt: 